## This notebook takes the database created by the raw scrape of game results, cleans up some problems with the data 

### Task List
- the advanced metrics tables should have the team name added to each player's row
    - can also probably store as a single table instead of two tables
        - would also want to add home or away to each row along with Team Name
        Should be able to do it with a rather simple if then and the team names in the Game_ID

- Import the master rosters that are scraped and stored as CSV into the database so we can join data on age, class rank, ect 
    - I 

In [6]:
## Dependencies

import pandas as pd
import numpy as np

import sqlite3

# db_path = '../data/2022-2023 Season Data.db' # Set FOr 2022-2023 Season

db_path = '../TEMP/2023_Season_Nov 2_Game_Stats.db' # Set For 2023-2024 Season


conn = sqlite3.connect(db_path)

# Roster data
folder = '../data/rosters/'

df_2023 = pd.read_csv(folder + '2023_master_roster.csv')
df_2022 = pd.read_csv(folder + '2022_master_roster.csv')
df_2021 = pd.read_csv(folder + '2021_master_roster.csv')
df_2020 = pd.read_csv(folder + '2020_master_roster.csv')


## Print tables in database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())




[('game_details',), ('scoring_summary',), ('penalty_summary',), ('goalie_stats',), ('player_stats',), ('line_chart',), ('linescore',), ('advanced_metrics_team1',), ('advanced_metrics_team2',)]


## Create dictionary of Team Primary Names to Abbreviations

- Needed to Add IVY teams becuase of low amount of games. will have to do for harvard, yale, ect next week


In [7]:
## Create dataframe from SQL query
df = pd.read_sql_query("SELECT * FROM linescore", conn)

# Function to count the occurrences of primary team names for unmatched abbreviations
def count_primary_names_for_abbreviation(abbreviation):
    filtered_rows = df[df['Team'] == abbreviation]
    team_counts = {}
    
    for _, row in filtered_rows.iterrows():
        teams = row['Game_ID'].split('-')[-2:]
        for team in teams:
            if team not in team_counts:
                team_counts[team] = 0
            team_counts[team] += 1
            
    return team_counts


# Attempt to match abbreviations to primary names based on substrings and common naming conventions
matched_dict = {}
unmatched_abbreviations = []

for abbreviation in df['Team'].unique():



# Match the abbreviation to the primary team name with the highest occurrence

    team_counts = count_primary_names_for_abbreviation(abbreviation)
    # Get the team with the highest count
    matched_team = max(team_counts, key=team_counts.get)
    matched_dict[abbreviation] = matched_team

# matched_dict

# Manually fix the unmatched abbreviations - IVY League Teams with no of very few games throw a wrench in the above method
# Brown: Brown
# Cornell: Cornell

# Make those substitutions
matched_dict['Brown'] = 'Brown'
matched_dict['Cornell'] = 'Cornell'
# yale
matched_dict['Yale'] = 'Yale'
# princeton
matched_dict['Princeton'] = 'Princeton'

# harvard
matched_dict['Harvard'] = 'Harvard'
# columbia
matched_dict['Columbia'] = 'Columbia'

# dartmouth
matched_dict['Dartmouth'] = 'Dartmouth'

# penn
matched_dict['Penn'] = 'Penn'

# BC
matched_dict['BC'] = 'Boston College'



matched_dict

{'LSSU': 'Lake Superior',
 'MSU': 'Michigan State',
 'RIT': 'RIT',
 'SLU': 'St. Lawrence',
 'Clarkson': 'Clarkson',
 'NDame': 'Notre Dame',
 'UConn': 'Connecticut',
 'Colgate': 'Colgate',
 'AIC': "American Int'l",
 'UMass': 'Massachusetts',
 'PC': 'Providence',
 'Michigan': 'Michigan',
 'Stonehill': 'Stonehill',
 'Northeastern': 'Northeastern',
 'OSU': 'Ohio State',
 'Mercyhurst': 'Mercyhurst',
 'PSU': 'Penn State',
 'LIU': 'Long Island',
 'Lindenwood': 'Lindenwood',
 'Air Force': 'Air Force',
 'BGSU': 'Bowling Green',
 'RMU': 'Robert Morris',
 'Augustana': 'Augustana',
 'Wisconsin': 'Wisconsin',
 'Denver': 'Denver',
 'Alaska': 'Alaska',
 'Union': 'Union',
 'Army': 'Army',
 'SCSU': 'St. Cloud State',
 'UST': 'St. Thomas',
 'Miami': 'Miami',
 'FSU': 'Ferris State',
 'HC': 'Holy Cross',
 'UML': 'Lowell',
 'UAA': 'Anchorage',
 'WMU': 'Western Michigan',
 'RPI': 'Rensselaer',
 'Maine': 'Maine',
 'UVM': 'Vermont',
 'SHU': 'Sacred Heart',
 'BC': 'Boston College',
 'Canisius': 'Canisius',
 'M

## Clean and Transform Advanced Metrics
- add, Team and Home-Away columns, combine the two tables into a single table

In [8]:
### NEW Simple - advanced metrics 1 and 2 now have a team column (still contains the secondary team name so will want to address that as well
## Combine the two tables into advanced_metrics_combined

# Get entire first table of advanced metrics
adv_met_1 = pd.read_sql_query("SELECT * FROM advanced_metrics_team1", conn)


# Get entire second table of advanced metrics
adv_met_2 = pd.read_sql_query("SELECT * FROM advanced_metrics_team2", conn)


# Combine the two tables
adv_met_combined = pd.concat([adv_met_1, adv_met_2], axis=0)

# Apply the matched_dict to the Team column
adv_met_combined['Team'] = adv_met_combined['Team'].apply(lambda x: matched_dict[x])


# Fill NaNs with 0
adv_met_combined = adv_met_combined.fillna(0)

# Add to database
adv_met_combined.to_sql('advanced_metrics_combined', conn, if_exists='replace', index=False)

# View Sample and output CSV
adv_met_combined.head()

# adv_met_combined.to_csv('../TEMP/NEWNEW advanced_metrics_combined.csv', index=False)

,Team,Player,TOTAL_Block,TOTAL_Miss,TOTAL_Saved,TOTAL_Goals,TOTAL_Total_Shots,EVEN_Block,EVEN_Miss,EVEN_Saved,...,PP_Goals,PP_Total_Shots,CLOSE_Block,CLOSE_Miss,CLOSE_Saved,CLOSE_Goals,CLOSE_Total_Shots,D_Blocks,Faceoffs,Game_ID
0,Lake Superior,Harrison Roy,1,2,,1,4,1,2,,...,,0,1,1,,,2,,4-15,2023-10-07-Lake Superior-Michigan State
1,Lake Superior,John Herrington,,,4,,4,,,4,...,,0,,,,,0,,,2023-10-07-Lake Superior-Michigan State
2,Lake Superior,Tyler Williams,,,3,1,4,,,3,...,,0,,,1,,1,,,2023-10-07-Lake Superior-Michigan State
3,Lake Superior,Artyom Borshyov,1,1,2,,4,1,1,2,...,,0,1,1,1,,3,,,2023-10-07-Lake Superior-Michigan State
4,Lake Superior,Grant Hindman,,1,2,,3,,1,2,...,,0,,1,1,,2,2,,2023-10-07-Lake Superior-Michigan State


## Add Home and Away Columns to game_details table

In [9]:
# Step 1: Read the game_details table into a DataFrame
df_game_details = pd.read_sql("SELECT * FROM game_details", conn)

# Step 2: Create new columns for Home and Away Teams by parsing Game_ID
df_game_details['Away_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[3])
df_game_details['Home_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[4])

# Step 3: Write this updated DataFrame back to the game_details table
df_game_details.to_sql('game_details', conn, if_exists='replace', index=False)


176

## Clean up The Column Names and extra header rows in the Player Stats table

In [10]:
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

# view sample
player_stats_df.head()

,Team,Player,G,A,Pt.,+/-,Sh,PIM,FOW,FOL,FO%,Game_ID
0,Michigan State,Michigan State,G,A,Pt.,+/-,Sh,PIM,NaN,NaN,NaN,2023-10-07-Lake Superior-Michigan State
1,Michigan State,Gavin O'Connell,0,1,1,2,1,0,NaN,NaN,NaN,2023-10-07-Lake Superior-Michigan State
2,Michigan State,Tommi Männistö,0,1,1,1,2,0,NaN,NaN,NaN,2023-10-07-Lake Superior-Michigan State
3,Michigan State,Maxim Štrbák,0,0,0,-1,3,0,NaN,NaN,NaN,2023-10-07-Lake Superior-Michigan State
4,Michigan State,Artyom Levshunov,0,0,0,2,5,2,NaN,NaN,NaN,2023-10-07-Lake Superior-Michigan State


In [11]:
# Define a dictionary for column renaming
column_renames = {
    'Pt.': 'Pts',
    '+/-': 'plus_minus'
}

# Rename columns based on the dictionary
player_stats_df.rename(columns=column_renames, inplace=True)


# Drop rows where Team name is in the Player column
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

# Print the length of the dataframe
len(player_stats_df)


7011

In [12]:
## Change the Column names to be easy to work with
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

if 'Pt.' in player_stats_df.columns:
    player_stats_df.rename(columns={'Pt.': 'Pts'}, inplace=True)
else:
    print("Column 'Pt.' not found.")

if '+/-' in player_stats_df.columns:
    player_stats_df.rename(columns={'+/-': 'plus_minus'}, inplace=True)
else:
    print("Column '+/-' not found.")

print(len(player_stats_df))

# Drop rows if Team name is in the player column
# If ['Team'] is the same as ['Player'] then drop that row
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

# add the dataframe back to the database
player_stats_df.to_sql('player_stats', conn, if_exists='replace', index=False)

# print(len(player_stats_df))
#################################
# player_stats_df.head()

7363


7011

# Check for other subsitu

In [13]:
## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame
df_linescores = pd.read_sql("SELECT * FROM linescore", conn)

# Apply the dictionary to the Team column
df_linescores['Team'] = df_linescores['Team'].apply(lambda x: matched_dict[x])

df_linescores.head()

,Team,goals1,goals2,goals3,goalsT,shots1,shots2,shots3,shotsT,Pen,PIM,PPG,PPO,FOW,FOL,FOW%,Game_ID
0,Lake Superior,0,2,0,2,8,11,12,31,2,4,0,2,20,45,30.769231,2023-10-07-Lake Superior-Michigan State
1,Michigan State,1,2,2,5,13,16,8,37,3,6,1,2,45,20,69.230769,2023-10-07-Lake Superior-Michigan State
2,RIT,0,1,2,3,8,15,6,29,6,12,0,3,30,38,44.117647,2023-10-07-RIT-St. Lawrence
3,St. Lawrence,1,0,3,4,12,7,15,34,3,6,1,6,38,30,55.882353,2023-10-07-RIT-St. Lawrence
4,Clarkson,1,0,2,3,8,6,11,25,4,8,0,3,28,30,48.275862,2023-10-07-Clarkson-Notre Dame


# Penalty Table & Scoring Summary

In [14]:
## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame

df_penalty = pd.read_sql("SELECT * FROM penalty_summary", conn)

# Apply the dictionary to the Team column
#$ Skip if not found

df_penalty['Team'] = df_penalty['Team'].apply(lambda x: matched_dict[x])

    

# Apply same method to scorring_summary:
df_scoring = pd.read_sql("SELECT * FROM scoring_summary", conn)
df_scoring['Team'] = df_scoring['Team'].apply(lambda x: matched_dict[x])


## Add each table back to database
# Write the updated linescores DataFrame back to the linescore table
df_linescores.to_sql('linescore', conn, if_exists='replace', index=False)

# Write the updated penalty DataFrame back to the penalty_summary table
df_penalty.to_sql('penalty_summary', conn, if_exists='replace', index=False)

# Write the updated scoring DataFrame back to the scoring_summary table
df_scoring.to_sql('scoring_summary', conn, if_exists='replace', index=False)

1052

## CREATE A NEW TABLE WITH AGGRIGATED PLAYER STATS YEAR TO DATE

In [15]:
# Use player_stats_df from here on, instead of running another SQL query.
df_player_stats = player_stats_df.copy()


# Clean up the name format in player_stats for easier matching
# Replace the non-breaking space with a regular space
df_player_stats['Clean_Player'] = df_player_stats['Player'].apply(lambda x: x.replace('\xa0', ' '))

# Remove rows where Player is the team name (e.g., "Michigan State")
df_player_stats_cleaned = df_player_stats[df_player_stats['Player'] != df_player_stats['Team']]

# Convert relevant columns to integers for correct aggregation
cols_to_convert = ['G', 'A', 'Pts', 'plus_minus', 'Sh', 'PIM']
for col in cols_to_convert:
    df_player_stats_cleaned[col] = pd.to_numeric(df_player_stats_cleaned[col], errors='coerce')

# Aggregate the data for year-to-date stats
# Add a column for counting the number of games each player has played
agg_player_stats_corrected_with_games = df_player_stats_cleaned.groupby(['Clean_Player', 'Team']).agg({
    'G': 'sum',
    'A': 'sum',
    'Pts': 'sum',
    'plus_minus': 'sum',
    'Sh': 'sum',
    'PIM': 'sum',
    'Game_ID': 'count'  # Counting the number of unique Game_IDs for each player
}).reset_index()

# Rename the Game_ID column to Games_Played
agg_player_stats_corrected_with_games.rename(columns={'Game_ID': 'Games_Played'}, inplace=True)

# Save the updated aggregated data back to the database, replacing the existing table
agg_player_stats_corrected_with_games.to_sql('player_stats_ytd', conn, if_exists='replace', index=False)

# Verify by loading some sample data from the updated table
sample_updated_ytd = pd.read_sql_query("SELECT * FROM player_stats_ytd LIMIT 5;", conn)
sample_updated_ytd


,Clean_Player,Team,G,A,Pts,plus_minus,Sh,PIM,Games_Played
0,A.J. Hodges,Bentley,2,1,3,-3,18,0,7
1,A.J. Macaulay,Alaska,0,1,1,-3,11,0,6
2,AJ Casperson,Long Island,0,0,0,0,0,0,1
3,Aaron Bohlinger,Massachusetts,1,3,4,-1,2,0,5
4,Aaron Grounds,Long Island,1,2,3,0,8,6,7


## Add the Roster data from the CSVs to the Database

In [16]:
################## SET THE ROSTER DATAFRAME TO THE CORRECT YEAR ####################
## MATCH THE DATAFRAME NAMES
df_master_roster = df_2023

## Season Year Value
season_year = 2023

# Clean up the name formats for joining
# Master roster: Convert "Last Name, First Name" to "First Name Last Name"
# df_master_roster['Clean_Name'] = df_master_roster['Player'].apply(lambda x: ' '.join(reversed(x.split(', '))))

# Rename Player to Clean_Name
df_master_roster.rename(columns={'Player': 'Clean_Name'}, inplace=True)
# Rename School to Team
df_master_roster.rename(columns={'School': 'Team'}, inplace=True)

# Clean up the Team column, remove '-' and replace with ' '
# df_master_roster['School'] = df_master_roster['Team'].apply(lambda x: x.replace('-', ' '))

## If there are an period in the column names, remove them
df_master_roster.columns = df_master_roster.columns.str.replace('.', '')



In [17]:
### Finally add the roster to the database as it's own table

df_master_roster['SeasonYear'] = season_year

# Save the roster data as a new table in the database
roster_table_name = 'master_roster'
df_master_roster.to_sql(roster_table_name, conn, if_exists='replace', index=False)
############################################################

# Verify by listing all the tables in the database again
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = conn.execute(tables_query).fetchall()
table_names_updated = [table[0] for table in tables]
table_names_updated


['goalie_stats',
 'line_chart',
 'advanced_metrics_team1',
 'advanced_metrics_team2',
 'advanced_metrics_combined',
 'game_details',
 'player_stats',
 'linescore',
 'penalty_summary',
 'scoring_summary',
 'player_stats_ytd',
 'master_roster']

## Save a backup of the transformed database and proceed to adding the roster info


In [1]:
# Output the current state of the database to a new SQLite file in the temp folder
db_cleaned_path = '../data/NEW_2023_YTD_Game_Stats_NEW_NEW.db'

## Save the database to a new file
conn_cleaned = sqlite3.connect(db_cleaned_path)







NameError: name 'sqlite3' is not defined